In [5]:
import numpy as np
import os
import sys
import argparse
import traceback


import logging
import numpy as np
import pandas as pd

from cmbnet.utils.utils_general import calculate_radiomics_features, calculate_synthseg_features
import logging
import numpy as np
import pandas as pd
import nibabel as nib
import glob
import ast

from scipy.ndimage import label as nd_label # to avoid conflict below

In [6]:
data_dir = "/home/cerebriu/data/datasets/processed_final"
datasets_mapping = {
    "DOU": {
        "folder": "cmb_dou"
    },
    "CRB": {
        "folder": "cmb_crb"
    },
    "rest": {
        "folder": "cmb_train"
    }
}

df_cmb_metadata = pd.read_csv("../../data-misc/csv/CMB_metadata_all.csv")
df_cmb_metadata.head(2)

/tmp/ipykernel_4140852/2501135080.py:14: DtypeWarning: Columns (7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cmb_metadata = pd.read_csv("../../data-misc/csv/CMB_metadata_all.csv")


,seriesUID,cmb_id,CM,size,radius,processed_id,RB_label,Location,Multiple,<5mm,Cause,Uncertain,Other,seq_type,res_level,field_strength,TE,subject,patientUID,Dataset
0,sMOMENI-264_T0_V3-sCMB-10,0,"[48, 127, 164]",116,3.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SWI,low,3,20.0,264_T0_MRI_SWI_BFC_50mm_HM_sCMB_V3,264mom,sMOMENI
1,sMOMENI-264_T0_V3-sCMB-10,1,"[69, 79, 194]",116,3.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SWI,low,3,20.0,264_T0_MRI_SWI_BFC_50mm_HM_sCMB_V3,264mom,sMOMENI


In [7]:
data_dir = '/storage/evo1/jorge/datasets/cmb'
cache_folder = '/storage/evo1/jorge/MicrobleedNet/notebooks/analysis/test.ipynb'
synthseg_dir = '/storage/evo1/jorge/datasets/cmb/synthseg_masks_resampled'

In [8]:
shape_data = []
for study in df_cmb_metadata['seriesUID'].unique():

        
        dataset = study.split("-")[0]

        # NOTE: to remove later
        if dataset not in datasets_mapping:
            # print(f"IGNORED study {study}")
            # return f"IGNORED study {study}"
            continue
        print(study)
        folder = datasets_mapping.get(dataset, datasets_mapping["rest"])
        study_folder = os.path.join(data_dir, folder["folder"], "Data", study)

        mri_file = os.path.join(study_folder, "MRIs", f"{study}.nii.gz")
        cmb_file = os.path.join(study_folder, "Annotations", f"{study}.nii.gz")
        synthseg_file = os.path.join(synthseg_dir, f"{study}_synthseg_resampled.nii.gz")

        mri_im = nib.load(mri_file)
        mri_data = mri_im.get_fdata()
        cmb_im = nib.load(cmb_file)
        cmb_data = cmb_im.get_fdata()
        synth_im = nib.load(synthseg_file)
        synth_data = synth_im.get_fdata()

        cmbs_df = df_cmb_metadata[df_cmb_metadata["seriesUID"] == study]

        labeled_array, num_features = nd_label(
            cmb_data == 1
        )  # Ensure we're labeling the correct regions
        results = []

        for i in range(1, num_features + 1):
            # print(f"label {i} .....")
            cmb_mask_individual = labeled_array == i
            hitted_CCs = []
            print(f"Size of mask {np.sum(cmb_mask_individual)}")

            for CM in cmbs_df["CM"]:
                CM = ast.literal_eval(CM)
                CM = tuple(map(int, CM))

                if cmb_mask_individual[CM]:
                    if CM not in hitted_CCs:
                        hitted_CCs.append(CM)
                        # # print(f"Found CMB for CC-{i} in CM-{CM}")
                        radiomics_results = calculate_radiomics_features(
                            mri_data, cmb_mask_individual
                        )
                        print(radiomics_results)
                        # synthseg_results = calculate_synthseg_features(
                        #     mri_data, cmb_mask_individual, synth_data
                        # )
                        # print(synthseg_results)
                        # results.append(
                        #     {"seriesUID": study, "CM": CM, **radiomics_results}
                        # )

            if not hitted_CCs:
                print(f"CC-{i} not found in metadata for study {study}")


DOU-15-CMB-13
Size of mask 121


KeyboardInterrupt: 

In [ ]:
shape_data_df = pd.DataFrame(shape_data)
shape_data_df['CM'] = shape_data_df['CM'].apply(lambda x: str(list(x)))
shape_data_df

,seriesUID,CM,shape_Elongation,shape_Flatness,shape_LeastAxisLength,shape_MajorAxisLength,shape_Maximum2DDiameterColumn,shape_Maximum2DDiameterRow,shape_Maximum2DDiameterSlice,shape_Maximum3DDiameter,...,glszm_SmallAreaHighGrayLevelEmphasis,glszm_SmallAreaLowGrayLevelEmphasis,glszm_ZoneEntropy,glszm_ZonePercentage,glszm_ZoneVariance,ngtdm_Busyness,ngtdm_Coarseness,ngtdm_Complexity,ngtdm_Contrast,ngtdm_Strength
0,DOU-15-CMB-13,"[87, 207, 191]",0.881830,0.780986,4.964397,6.356579,8.062258,7.071068,7.280110,8.062258,...,0.000068,0.000068,-3.203427e-16,0.008264,0.000000,0.000000,1000000.000000,0.000000,0.000000,0.000000
1,DOU-15-CMB-13,"[132, 211, 185]",0.944718,0.886505,5.568416,6.281314,8.062258,7.280110,7.280110,8.062258,...,0.000047,0.000047,-3.203427e-16,0.006849,0.000000,0.000000,1000000.000000,0.000000,0.000000,0.000000
2,DOU-15-CMB-13,"[141, 197, 117]",0.978490,0.864344,5.412936,6.262475,8.062258,6.708204,7.280110,8.124038,...,0.000048,0.000048,-3.203427e-16,0.006897,0.000000,0.000000,1000000.000000,0.000000,0.000000,0.000000
3,DOU-15-CMB-13,"[141, 204, 119]",0.797273,0.734729,4.293613,5.843807,6.324555,6.000000,6.708204,7.000000,...,0.000169,0.000169,-3.203427e-16,0.012987,0.000000,0.000000,1000000.000000,0.000000,0.000000,0.000000
4,DOU-15-CMB-13,"[161, 262, 306]",0.982816,0.980769,6.891050,7.026168,8.602325,8.602325,8.246211,9.000000,...,0.000018,0.000018,-3.203427e-16,0.004255,0.000000,0.000000,1000000.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,CRB-1.2.826.1.3680043.9.5282.150415.650.650232...,"[236, 146, 107]",0.350042,0.296265,4.039715,13.635501,12.649111,5.099020,12.369317,12.688578,...,1.058801,0.008389,2.584963e+00,0.032086,615.805556,2.187202,0.038443,5.519628,0.060885,0.269679
197,CRB-1.3.12.2.1107.5.2.6.24111.3000001908210412...,"[57, 226, 153]",0.872583,0.856583,4.841763,5.652413,7.211103,6.082763,6.708204,7.280110,...,8.919611,0.035496,1.879965e+00,0.102273,159.061728,1.982990,0.088755,2.614988,0.057063,0.358786
198,CRB-1.3.12.2.1107.5.2.6.24111.3000001908210412...,"[122, 256, 158]",0.404823,0.361536,4.249278,11.753404,11.045361,6.082763,15.524175,15.556349,...,3.745607,0.030993,3.321928e+00,0.069930,254.210000,1.027981,0.086942,4.943450,0.042708,1.068373
199,CRB-1.3.12.2.1107.5.2.6.24111.3000001908210412...,"[149, 282, 234]",0.342506,0.318889,4.007969,12.568535,12.041595,6.082763,15.524175,15.652476,...,4.861623,0.014649,3.169925e+00,0.072000,277.432099,1.222648,0.084633,3.259893,0.035377,0.643535


In [ ]:
enriched_CMB_data = pd.merge(df_cmb_metadata, shape_data_df, on=["seriesUID", "CM"], how="left")
enriched_CMB_data

,seriesUID,cmb_id,CM,size,radius,processed_id,RB_label,Location,Multiple,<5mm,...,glszm_SmallAreaHighGrayLevelEmphasis,glszm_SmallAreaLowGrayLevelEmphasis,glszm_ZoneEntropy,glszm_ZonePercentage,glszm_ZoneVariance,ngtdm_Busyness,ngtdm_Coarseness,ngtdm_Complexity,ngtdm_Contrast,ngtdm_Strength
0,sMOMENI-264_T0_V3-sCMB-10,0,"[48, 127, 164]",116,3.03,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sMOMENI-264_T0_V3-sCMB-10,1,"[69, 79, 194]",116,3.03,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sMOMENI-264_T0_V3-sCMB-10,2,"[116, 116, 60]",108,2.95,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,sMOMENI-264_T0_V3-sCMB-10,3,"[131, 59, 94]",98,2.86,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sMOMENI-264_T0_V3-sCMB-10,4,"[163, 64, 192]",56,2.37,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37764,CRB-1.2.826.1.3680043.9.5282.150415.650.650232...,12,"[236, 146, 107]",187,3.55,11.0,1.0,Cortex / grey-white junction,True,True,...,1.058801,0.008389,2.584963,0.032086,615.805556,2.187202,0.038443,5.519628,0.060885,0.269679
37765,CRB-1.3.12.2.1107.5.2.6.24111.3000001908210412...,0,"[57, 226, 153]",88,2.76,2.0,3.0,Cortex / grey-white junction,NaN,True,...,8.919611,0.035496,1.879965,0.102273,159.061728,1.982990,0.088755,2.614988,0.057063,0.358786
37766,CRB-1.3.12.2.1107.5.2.6.24111.3000001908210412...,1,"[122, 256, 158]",143,3.24,3.0,4.0,Basal ganglia grey matter,NaN,True,...,3.745607,0.030993,3.321928,0.069930,254.210000,1.027981,0.086942,4.943450,0.042708,1.068373
37767,CRB-1.3.12.2.1107.5.2.6.24111.3000001908210412...,2,"[149, 282, 234]",125,3.10,0.0,1.0,Cortex / grey-white junction,NaN,True,...,4.861623,0.014649,3.169925,0.072000,277.432099,1.222648,0.084633,3.259893,0.035377,0.643535


In [ ]:
enriched_CMB_data_filt = enriched_CMB_data[enriched_CMB_data['Dataset'].isin(["DOU", "CRB"])]
enriched_CMB_data_filt

,seriesUID,cmb_id,CM,size,radius,processed_id,RB_label,Location,Multiple,<5mm,...,glszm_SmallAreaHighGrayLevelEmphasis,glszm_SmallAreaLowGrayLevelEmphasis,glszm_ZoneEntropy,glszm_ZonePercentage,glszm_ZoneVariance,ngtdm_Busyness,ngtdm_Coarseness,ngtdm_Complexity,ngtdm_Contrast,ngtdm_Strength
37568,DOU-15-CMB-13,0,"[87, 207, 191]",121,3.07,NaN,NaN,NaN,NaN,NaN,...,0.000068,0.000068,-3.203427e-16,0.008264,0.000000,0.000000,1000000.000000,0.000000,0.000000,0.000000
37569,DOU-15-CMB-13,1,"[132, 211, 185]",146,3.27,NaN,NaN,NaN,NaN,NaN,...,0.000047,0.000047,-3.203427e-16,0.006849,0.000000,0.000000,1000000.000000,0.000000,0.000000,0.000000
37570,DOU-15-CMB-13,2,"[141, 197, 117]",145,3.26,NaN,NaN,NaN,NaN,NaN,...,0.000048,0.000048,-3.203427e-16,0.006897,0.000000,0.000000,1000000.000000,0.000000,0.000000,0.000000
37571,DOU-15-CMB-13,3,"[141, 204, 119]",77,2.64,NaN,NaN,NaN,NaN,NaN,...,0.000169,0.000169,-3.203427e-16,0.012987,0.000000,0.000000,1000000.000000,0.000000,0.000000,0.000000
37572,DOU-15-CMB-13,4,"[161, 262, 306]",235,3.83,NaN,NaN,NaN,NaN,NaN,...,0.000018,0.000018,-3.203427e-16,0.004255,0.000000,0.000000,1000000.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37764,CRB-1.2.826.1.3680043.9.5282.150415.650.650232...,12,"[236, 146, 107]",187,3.55,11.0,1.0,Cortex / grey-white junction,True,True,...,1.058801,0.008389,2.584963e+00,0.032086,615.805556,2.187202,0.038443,5.519628,0.060885,0.269679
37765,CRB-1.3.12.2.1107.5.2.6.24111.3000001908210412...,0,"[57, 226, 153]",88,2.76,2.0,3.0,Cortex / grey-white junction,NaN,True,...,8.919611,0.035496,1.879965e+00,0.102273,159.061728,1.982990,0.088755,2.614988,0.057063,0.358786
37766,CRB-1.3.12.2.1107.5.2.6.24111.3000001908210412...,1,"[122, 256, 158]",143,3.24,3.0,4.0,Basal ganglia grey matter,NaN,True,...,3.745607,0.030993,3.321928e+00,0.069930,254.210000,1.027981,0.086942,4.943450,0.042708,1.068373
37767,CRB-1.3.12.2.1107.5.2.6.24111.3000001908210412...,2,"[149, 282, 234]",125,3.10,0.0,1.0,Cortex / grey-white junction,NaN,True,...,4.861623,0.014649,3.169925e+00,0.072000,277.432099,1.222648,0.084633,3.259893,0.035377,0.643535


In [ ]:
enriched_CMB_data_filt[enriched_CMB_data_filt['Dataset'].isin(["DOU", "CRB"])]
enriched_CMB_data_filt.to_csv("../../data-misc/csv/CMB_metadata_all_enriched_filt.csv", index=False)